<a href="https://colab.research.google.com/github/ldessemon2/myailearning/blob/main/bb_error_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.cloud import bigquery
import pandas as pd
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# Replace with your BigQuery project ID
PROJECT_ID = 'internal-naoxyatrainingservice'
VIEW_NAME = 'oxyaca.beebole_all_view'

In [20]:
def fetch_timesheet_data(project_id: str, view_name: str) -> pd.DataFrame:
    # ... (same as before)
    client = bigquery.Client(project=project_id)
    query = f"""
        SELECT
            person_name,
            CAST(hours AS FLOAT64) AS hours,
            CAST(billableHours AS FLOAT64) AS billableHours,
            project_name,
            subproject_name,
            task_name,
            CAST(nonBillableHours AS FLOAT64) AS nonBillableHours,
            CAST(month AS INT64) AS month,
            CAST(year AS INT64) AS year,
            CAST(week AS INT64) AS week,
            # Directly select the 'date' column, assuming it's already a DATE type
            date
        FROM
            `{view_name}`
    """
    df = client.query(query).to_dataframe()
    return df

In [7]:
def calculate_weekly_features(df: pd.DataFrame) -> pd.DataFrame:
    # ... (same as before)
    weekly_data = df.groupby(['person_name', 'year', 'week']).agg(
        total_hours=('hours', 'sum'),
        total_billable_hours=('billableHours', 'sum'),
        total_non_billable_hours=('nonBillableHours', 'sum'),
        distinct_projects=('project_name', 'nunique'),
        distinct_tasks=('task_name', 'nunique'),
        first_day_of_week=('date', 'min'),
        last_day_of_week=('date', 'max')
    ).reset_index()

    weekly_data['billable_ratio'] = weekly_data['total_billable_hours'] / weekly_data['total_hours']
    weekly_data['non_billable_ratio'] = weekly_data['total_non_billable_hours'] / weekly_data['total_hours']
    weekly_data.replace([float('inf'), float('-inf'), pd.NA], 0, inplace=True)

    def count_days_worked(row):
        if row['first_day_of_week'] and row['last_day_of_week']:
            return (row['last_day_of_week'] - row['first_day_of_week']).days + 1
        return 0

    weekly_data['days_worked'] = weekly_data.apply(count_days_worked, axis=1)
    weekly_data['avg_hours_per_day'] = weekly_data['total_hours'] / weekly_data['days_worked'].replace(0, 1)

    return weekly_data

In [8]:
def identify_potential_fraud(weekly_df: pd.DataFrame) -> pd.DataFrame:
    # ... (same as before)
    weekly_df['fraud_flag'] = 0
    weekly_df.loc[weekly_df['total_hours'] > 60, 'fraud_flag'] = 1
    weekly_df.loc[weekly_df['distinct_projects'] > 5, 'fraud_flag'] = 1
    weekly_df.loc[weekly_df['distinct_tasks'] > 10, 'fraud_flag'] = 1
    weekly_df.loc[(weekly_df['billable_ratio'] < 0.1) & (weekly_df['total_hours'] > 10), 'fraud_flag'] = 1
    weekly_df.loc[(weekly_df['billable_ratio'] > 0.95) & (weekly_df['total_hours'] > 10), 'fraud_flag'] = 1
    weekly_df.loc[weekly_df['avg_hours_per_day'] > 12, 'fraud_flag'] = 1
    weekly_df.loc[weekly_df['avg_hours_per_day'] < 2, 'fraud_flag'] = 1
    return weekly_df

In [9]:
def simple_anomaly_detection(weekly_df: pd.DataFrame) -> pd.DataFrame:
    # ... (same as before)
    weekly_df['anomaly_flag'] = 0
    for employee in weekly_df['person_name'].unique():
        employee_data = weekly_df[weekly_df['person_name'] == employee]
        mean_hours = employee_data['total_hours'].mean()
        std_hours = employee_data['total_hours'].std()
        if std_hours > 0:
            threshold = 2
            weekly_df.loc[
                (weekly_df['person_name'] == employee) &
                ((weekly_df['total_hours'] - mean_hours) / std_hours > threshold),
                'anomaly_flag'
            ] = 1
            weekly_df.loc[
                (weekly_df['person_name'] == employee) &
                ((mean_hours - weekly_df['total_hours']) / std_hours > threshold),
                'anomaly_flag'
            ] = 1
    return weekly_df


In [26]:

def advanced_feature_engineering(weekly_df: pd.DataFrame) -> pd.DataFrame:
    """Creates more advanced features for fraud detection."""

    # Sort data by employee and week
    weekly_df = weekly_df.sort_values(by=['person_name', 'year', 'week'])

    # 1. Rate of change in total hours compared to the previous week
    weekly_df['prev_week_total_hours'] = weekly_df.groupby('person_name')['total_hours'].shift(1)
    weekly_df['hours_change_rate'] = (weekly_df['total_hours'] - weekly_df['prev_week_total_hours']) / weekly_df['prev_week_total_hours']
    weekly_df['hours_change_rate'].replace([float('inf'), float('-inf'), pd.NA], 0, inplace=True)

    # 2. Rate of change in number of distinct projects
    weekly_df['prev_week_distinct_projects'] = weekly_df.groupby('person_name')['distinct_projects'].shift(1)
    weekly_df['project_change_rate'] = (weekly_df['distinct_projects'] - weekly_df['prev_week_distinct_projects']) / weekly_df['prev_week_distinct_projects']
    weekly_df['project_change_rate'].replace([float('inf'), float('-inf'), pd.NA], 0, inplace=True)

    # 3. Ratio of billable hours to total hours (already calculated but can be used)

    # 4. Average hours worked on the same project within a week
    def avg_hours_per_project(group):
        total_hours_worked = group['hours'].sum()
        num_projects = group['project_name'].nunique()
        return total_hours_worked / num_projects if num_projects > 0 else 0

    project_avg_hours = timesheet_df.groupby(['person_name', 'year', 'week', 'project_name']).agg(
        hours=('hours', 'sum')
    ).reset_index()
    project_avg_hours['avg_project_hours'] = project_avg_hours.groupby(['person_name', 'year', 'week'])['hours'].transform(lambda x: x.sum() / x.count() if x.count() > 0 else 0)
    weekly_df = pd.merge(weekly_df, project_avg_hours[['person_name', 'year', 'week', 'avg_project_hours']].drop_duplicates(subset=['person_name', 'year', 'week']), on=['person_name', 'year', 'week'], how='left')

    # 5. Flag for working on an unusually high number of projects in a single day (requires daily granularity)
    # ----> Changed Code Block <----
    # Merge timesheet_df with weekly_df to get year and week info for daily data
    daily_data_with_week_info = pd.merge(timesheet_df[['person_name', 'date', 'year', 'week', 'project_name']], weekly_df[['person_name', 'year', 'week']].drop_duplicates(), on=['person_name', 'year', 'week'], how='inner')

    daily_project_counts = daily_data_with_week_info.groupby(['person_name', 'date'])['project_name'].nunique().reset_index(name='daily_project_count')

    # Now group by person_name, year, and week
    weekly_df = pd.merge(weekly_df, daily_project_counts.groupby(['person_name', 'year', 'week'])['daily_project_count'].max().reset_index(name='max_daily_projects'), on=['person_name', 'year', 'week'], how='left')

    return weekly_df

In [11]:
def train_anomaly_detection_model(weekly_features_df: pd.DataFrame):
    """Trains an anomaly detection model (Isolation Forest)."""
    # Select features for the model (exclude non-numeric or identifier columns)
    features = ['total_hours', 'total_billable_hours', 'total_non_billable_hours',
                'distinct_projects', 'distinct_tasks', 'billable_ratio',
                'non_billable_ratio', 'days_worked', 'avg_hours_per_day',
                'hours_change_rate', 'project_change_rate', 'avg_project_hours', 'max_daily_projects']
    X = weekly_features_df[features].fillna(weekly_features_df[features].mean()) # Handle missing values

    # Train Isolation Forest model
    model = IsolationForest(contamination='auto', random_state=42)
    model.fit(X)

    # Get anomaly scores and predictions (-1 for anomaly, 1 for inlier)
    weekly_features_df['anomaly_score'] = model.decision_function(X)
    weekly_features_df['anomaly_prediction'] = model.predict(X)
    weekly_features_df['is_anomaly'] = weekly_features_df['anomaly_prediction'].apply(lambda x: 1 if x == -1 else 0)

    return weekly_features_df, model

In [12]:
def train_fraud_classification_model(weekly_features_df: pd.DataFrame, labels: pd.Series):
    """Trains a fraud classification model (Random Forest).

    Args:
        weekly_features_df: DataFrame with weekly features.
        labels: Pandas Series containing the fraud labels (1 for fraud, 0 for not fraud).
    """
    features = ['total_hours', 'total_billable_hours', 'total_non_billable_hours',
                'distinct_projects', 'distinct_tasks', 'billable_ratio',
                'non_billable_ratio', 'days_worked', 'avg_hours_per_day',
                'hours_change_rate', 'project_change_rate', 'avg_project_hours', 'max_daily_projects']
    X = weekly_features_df[features].fillna(weekly_features_df[features].mean())
    y = labels

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Train Random Forest classifier
    model = RandomForestClassifier(random_state=42, class_weight='balanced') # Consider class imbalance
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print("\nFraud Classification Model Evaluation:")
    print(classification_report(y_test, y_pred))
    print(f"AUC: {roc_auc_score(y_test, y_proba)}")

    # Plot ROC Curve
    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_score(y_test, y_proba):.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

    # Plot Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    average_precision = average_precision_score(y_test, y_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='blue', lw=2, label=f'Precision-Recall curve (AP = {average_precision:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.show()

    return model

In [13]:
def deploy_model(model, weekly_data: pd.DataFrame, features: list):
    """Illustrative function for deploying the model (can be adapted)."""
    X_new = weekly_data[features].fillna(weekly_data[features].mean())
    if hasattr(model, 'predict_proba'):
        predictions = model.predict_proba(X_new)[:, 1] # Get probability of fraud
    else:
        predictions = model.decision_function(X_new) # Get anomaly scores

    # You would typically integrate this with a system to flag or review these predictions
    weekly_data['fraud_probability'] = predictions
    potential_fraud = weekly_data[weekly_data['fraud_probability'] > 0.7] # Example threshold

    print("\nPotential Fraudulent Weeks (based on deployed model):")
    print(potential_fraud[['person_name', 'year', 'week', 'fraud_probability']])

In [18]:
from google.colab import auth
auth.authenticate_user()

In [21]:
# 1. Fetch data from BigQuery
print("Fetching data from BigQuery...")
timesheet_df = fetch_timesheet_data(PROJECT_ID, VIEW_NAME)
print("Data fetched successfully.")

Fetching data from BigQuery...
Data fetched successfully.


In [22]:
# 2. Calculate weekly features
print("Calculating weekly features...")
weekly_features_df = calculate_weekly_features(timesheet_df)
print("Weekly features calculated.")

Calculating weekly features...
Weekly features calculated.


In [23]:
# 3. Identify potential fraud using rule-based approach
print("Identifying potential fraud using rule-based approach...")
fraud_identified_df = identify_potential_fraud(weekly_features_df.copy())
potential_fraud_weeks_rule_based = fraud_identified_df[fraud_identified_df['fraud_flag'] == 1]
if not potential_fraud_weeks_rule_based.empty:
    print("\nPotential fraudulent weekly timesheets (Rule-based):")
    print(potential_fraud_weeks_rule_based)
else:
    print("\nNo potential fraudulent weekly timesheets found based on the defined rules.")

Identifying potential fraud using rule-based approach...

Potential fraudulent weekly timesheets (Rule-based):
             person_name  year  week  total_hours  total_billable_hours  \
1      Abdelkader Kechad  2021    13         40.0                  30.5   
3      Abdelkader Kechad  2021    15         41.5                  36.5   
4      Abdelkader Kechad  2021    16         40.0                  34.0   
5      Abdelkader Kechad  2021    17         40.0                  26.0   
6      Abdelkader Kechad  2021    18         40.0                  34.5   
...                  ...   ...   ...          ...                   ...   
11451       Yazid Hocine  2024    14         40.0                  34.5   
11452       Yazid Hocine  2024    15         40.0                  34.5   
11453       Yazid Hocine  2024    16         40.0                  33.5   
11454       Yazid Hocine  2024    17         41.5                  33.0   
11455       Yazid Hocine  2024    18         40.0               

In [24]:
# 4. Implement a simple anomaly detection based on total hours
print("\nPerforming simple anomaly detection based on total hours...")
anomaly_detected_df_simple = simple_anomaly_detection(weekly_features_df.copy())
anomalous_weeks_simple = anomaly_detected_df_simple[anomaly_detected_df_simple['anomaly_flag'] == 1]
if not anomalous_weeks_simple.empty:
    print("\nAnomalous weekly timesheets (Simple Anomaly Detection):")
    print(anomalous_weeks_simple)
else:
    print("\nNo anomalous weekly timesheets found based on the simple anomaly detection.")


Performing simple anomaly detection based on total hours...

Anomalous weekly timesheets (Simple Anomaly Detection):
             person_name  year  week  total_hours  total_billable_hours  \
60     Abdelkader Kechad  2022    20         60.5                  59.0   
61     Abdelkader Kechad  2022    21         53.0                  44.0   
108    Abdelkader Kechad  2023    16         55.0                  53.0   
115    Abdelkader Kechad  2023    23         55.0                  54.0   
118    Abdelkader Kechad  2023    26         53.0                  43.0   
...                  ...   ...   ...          ...                   ...   
11416       Yazid Hocine  2023    32         46.5                  44.5   
11417       Yazid Hocine  2023    33         45.0                  31.0   
11425       Yazid Hocine  2023    41         54.5                  45.5   
11427       Yazid Hocine  2023    43         48.0                  48.0   
11450       Yazid Hocine  2024    13         48.5        

In [27]:
# 5. Advanced Feature Engineering
print("\nPerforming advanced feature engineering...")
weekly_features_df_advanced = advanced_feature_engineering(weekly_features_df.copy())
print("Advanced features engineered.")


Performing advanced feature engineering...


<ipython-input-26-1aee02e7ecb5>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weekly_df['hours_change_rate'].replace([float('inf'), float('-inf'), pd.NA], 0, inplace=True)
<ipython-input-26-1aee02e7ecb5>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 

KeyError: 'year'

In [ ]:
# 6. Model Selection and Training (Example: Anomaly Detection with Isolation Forest)
    print("\nTraining Anomaly Detection Model (Isolation Forest)...")
    weekly_features_df_with_anomalies, anomaly_model = train_anomaly_detection_model(weekly_features_df_advanced.copy())
    anomalous_weeks_advanced = weekly_features_df_with_anomalies[weekly_features_df_with_anomalies['is_anomaly'] == 1]
    if not anomalous_weeks_advanced.empty:
        print("\nAnomalous weekly timesheets (Isolation Forest):")
        print(anomalous_weeks_advanced[['person_name', 'year', 'week', 'anomaly_score', 'anomaly_prediction']])
    else:
        print("\nNo anomalies detected by the Isolation Forest model.")

In [ ]:
# 7. Model Selection and Training (Example: Fraud Classification with Random Forest - Requires Labeled Data)
    # **Important:** To run this part, you need a column in your `weekly_features_df_advanced` that indicates whether a week was actually fraudulent (e.g., a 'is_fraud' column with 0 or 1).
    if 'is_fraud' in weekly_features_df_advanced.columns:
        print("\nTraining Fraud Classification Model (Random Forest)...")
        fraud_model = train_fraud_classification_model(weekly_features_df_advanced.copy(), weekly_features_df_advanced['is_fraud'])

        # 8. Deployment (Illustrative)
        print("\nIllustrative Deployment of Fraud Classification Model...")
        features_for_model = ['total_hours', 'total_billable_hours', 'total_non_billable_hours',
                                'distinct_projects', 'distinct_tasks', 'billable_ratio',
                                'non_billable_ratio', 'days_worked', 'avg_hours_per_day',
                                'hours_change_rate', 'project_change_rate', 'avg_project_hours', 'max_daily_projects']
        deploy_model(fraud_model, weekly_features_df_advanced.copy(), features_for_model)
    else:
        print("\nSkipping Fraud Classification Model training as no 'is_fraud' label column found.")

    print("\nError fraud detection process completed with advanced steps.")